In [1]:
#역과 관측소를 위도와 경도 데이터를 이용해 매핑

import pandas as pd


f = open("역.txt", 'r')
station_name = f.readlines()
f = open("관측소.txt", 'r')
aws_num = f.readlines()

station_df=pd.read_csv('서울교통공사_1_8호선 역사 좌표(위경도) 정보_20241031.csv',encoding='cp949')
aws_df=pd.read_csv('META_관측지점정보_20250528115343.csv',encoding='cp949')



In [2]:
#서울 바깥의 역들 거거
station_df=station_df[['역명','위도','경도']]
out =['산성','남한산성입구','단대오거리','신흥','수진','모란','하남검단산','하남시청','하남풍산'
     ,'미사','도봉산','장암','구산','독바위','연신내','구파발','지축','디지털미디어시티','응암','새절'
     ,'증산','방화','온수','천왕','고덕']
station_df = station_df[~station_df['역명'].isin(out)]

In [3]:
aws_df=aws_df[aws_df["종료일"].isna()]
aws_df=aws_df[['지점','지점명','위도','경도']]
aws_num=[int(x.strip()) for x in aws_num]
aws_df=aws_df[aws_df['지점'].isin(aws_num)]


In [4]:
aws_df.head()

,지점,지점명,위도,경도
434,400,강남,37.4982,127.0816
439,401,서초,37.4846,127.0260
443,402,강동,37.5556,127.1450
447,403,송파,37.4743,127.1254
451,404,강서,37.5739,126.8295


In [5]:
from geopy.distance import geodesic


def find_nearest_station(subway_df, station_df):
    nearest_station_names = []

    for _, subway_row in subway_df.iterrows():
        subway_coord = (subway_row['위도'], subway_row['경도'])

        # 거리 계산
        distances = station_df.apply(
            lambda row: geodesic(subway_coord, (row['위도'], row['경도'])).meters,
            axis=1
        )

        # 가장 가까운 관측소 인덱스
        min_index = distances.idxmin()
        nearest_station = station_df.loc[min_index, '지점명']

        nearest_station_names.append(nearest_station)

    # 새로운 칼럼으로 추가
    subway_df['가장_가까운_관측소명'] = nearest_station_names
    return subway_df

result=find_nearest_station(station_df, aws_df)
result.head()
result.to_csv('지하철역별_가까운_관측소.csv',encoding="utf-8-sig",index=False)

In [7]:
import folium
import json

# 서울 중심 좌표
seoul_center = [37.5665, 126.9780]

# 지도 생성
m = folium.Map(location=seoul_center, zoom_start=11)

# GeoJSON 파일 로드
with open('kr.json', 'r', encoding='utf-8') as f:
    geo_data = json.load(f)

# 서울시 경계만 필터링
seoul_geo = {
    "type": "FeatureCollection",
    "features": [feature for feature in geo_data['features'] if feature['properties']['name'] == 'Seoul']
}

# 서울시 경계 추가
folium.GeoJson(
    seoul_geo,
    name='Seoul Boundary',
    style_function=lambda x: {
        'fillColor': '#00000000',
        'color': 'red',
        'weight': 4
    }
).add_to(m)

# 지도 저장
m.save('seoul_boundary_map.html')
